## Test guppy basecalling on REDFlesh_ON_Cas9_enrichment_data

RedFlesh apple Type1 raw fast5 ONT data 

Was initially basecalled using albacore2 using this notebook https://jupyterhub.powerplant.pfr.co.nz/user/hrpelg/notebooks/hrpelg/Red_Flesh_ON/RedFlesh_ON_Cas9_enrichment_ON_run_1_Base_calling_QC.ipynb#

🍂🍂🍂🍂🍂🍂🍂🍂🍂🍂🍂🍂🍂🍂🍂🍂🍂🍂🍂🍂🍂🍂🍂🍂🍂🍂🍂🍂🍂🍂🍂🍂🍂🍂🍂🍂🍂🍂🍂🍂🍂🍂🍂🍂🍂🍂🍂🍂🍂🍂


**Author: Elena Lopez-Girona **

In [15]:
module load guppy/3.2.4

In [95]:
module purge

In [2]:
INDIR1=/input/genomic/plant/Malus/OxfordNanopore/Red_flesh
WKDIR=/workspace/$USER/github/analysis-workflows/Malus/Red_Flesh_ON

# reference apple genome
INFASTA=/input/genomic/plant/Malus/Genome/GDDH_v1.1/Decompress/GDDH13_1-1_formatted.fasta 

In [13]:
mkdir $WKDIR/Guppy_basecalling/log

In [21]:
mkdir $WKDIR/Guppy_basecalling/00.Fastq

In [14]:
ls $INPUT

FAK77153_381a627fa445f6ab43dcbb0542d4cc06427d04b3_0.fast5
FAK77153_381a627fa445f6ab43dcbb0542d4cc06427d04b3_1.fast5


In [16]:
bsub << EOF
#!/bin/bash
#BSUB -J guppy
#BSUB -o $WKDIR/Guppy_basecalling/log/guppy.out 
#BSUB -e $WKDIR/Guppy_basecalling/log/guppy.err
#BSUB -R "gpu"

guppy_basecaller --input_path $INPUT --save_path $WKDIR/Guppy_basecalling/00.Fastq \
--flowcell FLO-MIN106 --kit SQK-LSK109 -x "cuda:0"

EOF

Job <955580> is submitted to default queue <normal>.


# Comparison between quality of  fastq raw reads between albacore2 and Guppy basecalling

In [62]:
module purge

In [81]:
module load FastQC/0.11.7

In [35]:
mkdir $WKDIR/albacore2/FastQC


* First look for quality of albacore2 basecalled reads ( I dind't do it in the the other notebook:\
/Red_Flesh_ON/RedFlesh_ON_Cas9_enrichment_ON_run_1_Base_calling_QC.ipynb')

In [78]:
#mkdir $WKDIR/Guppy_basecalling/FastQC

In [84]:
bsub << EOF
#!/bin/bash
#BSUB -J fastqc
#BSUB -o $WKDIR/albacore2/FastQC/fastqc.out
#BSUB -e $WKDIR/albacore2/FastQC/fastqc.err
#BSUB -n 24
#BSUB -R "span[hosts=1]"

fastqc -t 24 -q -o $WKDIR/albacore2/FastQC/ $WKDIR/albacore2/All_DS_RedFlesh_ON_run1_cas.fastq 

EOF

Job <958353> is submitted to default queue <normal>.


* Second look for quality of rawreads after basecalling with Guppy

In [71]:
bsub << EOF
#!/bin/bash
#BSUB -J fastqc
#BSUB -o $WKDIR/Guppy_basecalling/fastqc.out
#BSUB -e $WKDIR/Guppy_basecalling/fastqc.err
#BSUB -n 24
#BSUB -R "span[hosts=1]"


fastqc -t 24 -q -o $WKDIR/FastQC $WKDIR/Guppy_basecalling/00.Fastq/*.fastq

EOF

Job <958325> is submitted to default queue <normal>.


# pycoQC

In [96]:

module load conda

(base) 

: 1

In [97]:
conda activate cflsjt_albacore

(/workspace/appscratch/miniconda/cflsjt_albacore) 

: 1

In [24]:
mkdir -p $WKDIR/Guppy_basecalling/01.pycoQC/log

In [102]:
bsub -o $WKDIR/01.pycoQC/log/QC1.out -e $WKDIR/01.pycoQC/log/QC1.err -J ONT_QC1 "pycoQC -f $WKDIR/sequencing_summary.txt $WKDIR/01.pycoQC/pycoQC_summary_run1"

Job <958367> is submitted to default queue <normal>.
(/workspace/appscratch/miniconda/cflsjt_albacore) 

: 1

In [ ]:
#Some stats with Nanopack

In [ ]:
NanoPlot --summary $WKDIR/albacore2/Red_flesh_ON_run1_Cas9/all_summary_run1.txt --loglength -o $WKDIR/albacore2/Red_flesh_ON_run1_Cas9/summary-plots-log-transformed 

# Trim using porechop

In [111]:
conda deactivate

(base) 

: 1

In [112]:
conda activate cflsjt_porechop

(/workspace/appscratch/miniconda/cflsjt_porechop) 

: 1

In [113]:
module load htslib/1.7

(/workspace/appscratch/miniconda/cflsjt_porechop) 

: 1

In [107]:
mkdir -p $WKDIR/02.poreChop/log

(/workspace/appscratch/miniconda/cflsjt_porechop) 

: 1

* Merge fastq files

* Merge the fastqs from the various sub dirs before chopping

In [109]:
bsub -o $WKDIR/02.poreChop/log/merge.out -e $WKDIR/02.poreChop/log/merge.err -J merge \
"cat $WKDIR/00.Fastq/*.fastq | bgzip -c > $WKDIR/02.poreChop/Merged_RedFlesh_ON_GUPPY_cas.fastq.gz"

Job <958369> is submitted to default queue <normal>.
(/workspace/appscratch/miniconda/cflsjt_porechop) 

: 1

In [115]:
bsub << EOF
#!/bin/bash
#BSUB -J Porechop
#BSUB -o $WKDIR/02.poreChop/log/chop1.out 
#BSUB -e $WKDIR/02.poreChop/log/chop1.err
#BSUB -n 12
#BSUB -R "span[hosts=1]"

porechop -i $WKDIR/02.poreChop/Merged_RedFlesh_ON_GUPPY_cas.fastq.gz \
-o $WKDIR/02.poreChop/After_PoreCHOP_RedFlesh_ON_GUPPY_cas.fastq.gz \
-t 12 --discard_middle

EOF

Job <958374> is submitted to default queue <normal>.
(/workspace/appscratch/miniconda/cflsjt_porechop) 

: 1

7,056 reads loaded


Trimming adapters from read ends:

* SQK-NSK007_Y_Top: AATGTACTTCGTTCAGTTACGTATTGCTESC
* SQK-NSK007_Y_Bottom: GCAATACGTAACTGAACGAAGTESC
* SQK-MAP006_Short_Y_Top_LI32: CGGCGTCTGCTTGGGTGTTTAACCTESC
* SQK-MAP006_Short_Y_Bottom_LI33: GGTTAAACACCCAAGCAGACGCCGESC


* 784 / 7,056 reads had adapters trimmed from their start (12,489 bp removed)
* 1,087 / 7,056 reads had adapters trimmed from their end (13,031 bp removed)
* 1 / 7,056 reads were discarded based on middle adapters



# Create Metric plots (pyQC min read quality 7) and porechop using Nanopack

In [6]:
module purge

In [1]:
module load nanopack/1.0.0

In [3]:
NanoPlot --listcolors 

aliceblue, antiquewhite, aqua, aquamarine, azure, beige, bisque, black, blanchedalmond, blue, blueviolet, brown, burlywood, cadetblue, chartreuse, chocolate, coral, cornflowerblue, cornsilk, crimson, cyan, darkblue, darkcyan, darkgoldenrod, darkgray, darkgreen, darkgrey, darkkhaki, darkmagenta, darkolivegreen, darkorange, darkorchid, darkred, darksalmon, darkseagreen, darkslateblue, darkslategray, darkslategrey, darkturquoise, darkviolet, deeppink, deepskyblue, dimgray, dimgrey, dodgerblue, firebrick, floralwhite, forestgreen, fuchsia, gainsboro, ghostwhite, gold, goldenrod, gray, green, greenyellow, grey, honeydew, hotpink, indianred, indigo, ivory, khaki, lavender, lavenderblush, lawngreen, lemonchiffon, lightblue, lightcoral, lightcyan, lightgoldenrodyellow, lightgray, lightgreen, lightgrey, lightpink, lightsalmon, lightseagreen, lightskyblue, lightslategray, lightslategrey, lightsteelblue, lightyellow, lime, limegreen, linen, magenta, maroon, mediumaquamarine, mediumblue, mediumorc

In [ ]:
NanoPlot -c pink -f tiff --title Guppy --N50 -p Guppy_nanoplot_all_reads -o $WKDIR/02.poreChop/ --fastq $WKDIR/02.poreChop/After_PoreCHOP_RedFlesh_ON_GUPPY_cas.fastq.gz 

In [36]:
NanoComp -c darkcyan pink -p Albacore2_vs_Guppy_Canu_corrected_reads -o $WKDIR/02.poreChop/ --title Albacore2_vs_Guppy_reads -n Albacore2 Guppy --fastq $WKDIR/02.poreChop/All_DS_RedFlesh_ON_run1_cas_after_porechop_dis.fastq.gz $WKDIR/Guppy_basecalling/02.poreChop/After_PoreCHOP_RedFlesh_ON_GUPPY_cas.fastq.gz

# Align Reads to apple GDv1.1 using minimap2

In [1]:
module purge
module load minimap2/2.9
module load samtools

In [49]:
FQ=$WKDIR/Guppy_basecalling/02.poreChop/After_PoreCHOP_RedFlesh_ON_GUPPY_cas.fastq.gz 

In [120]:
mkdir -p $WKDIR/03.minimap2/log

In [122]:
bsub << EOF
#!/bin/bash
#BSUB -J minimap2
#BSUB -o $WKDIR/03.minimap2/log/map.out 
#BSUB -e $WKDIR/03.minimap2/log/map.err
#BSUB -n 4
#BSUB -R "span[hosts=1]"

minimap2 -ax map-ont $INFASTA $FQ | \
samtools view -Su - | samtools sort - -o $WKDIR/03.minimap2/Guppy_RedFlesh.bam

EOF

Job <958735> is submitted to default queue <normal>.


In [123]:
bsub << EOF
#!/bin/bash
#BSUB -J minimap2
#BSUB -o $WKDIR/03.minimap2/log/index_depth.out 
#BSUB -e $WKDIR/03.minimap2/log/index_depth.err
#BSUB -n 1
#BSUB -R "span[hosts=1]"

samtools index $WKDIR/03.minimap2/Guppy_RedFlesh.bam
samtools depth $WKDIR/03.minimap2/Guppy_RedFlesh.bam > $WKDIR/03.minimap2/Guppy_RedFlesh.bam.coverage
EOF

Job <958824> is submitted to default queue <normal>.


In [4]:
module load samtools/1.9
samtools stats $WKDIR/03.minimap2/Guppy_RedFlesh.bam | grep 'SN'

# Summary Numbers. Use `grep ^SN | cut -f 2-` to extract this part.
SN	raw total sequences:	7052
SN	filtered sequences:	0
SN	sequences:	7052
SN	is sorted:	1
SN	1st fragments:	7052
SN	last fragments:	0
SN	reads mapped:	6448
SN	reads mapped and paired:	0	# paired-end technology bit set + both mates mapped
SN	reads unmapped:	604
SN	reads properly paired:	0	# proper-pair bit set
SN	reads paired:	0	# paired-end technology bit set
SN	reads duplicated:	0	# PCR or optical duplicate bit set
SN	reads MQ0:	31	# mapped and MQ=0
SN	reads QC failed:	0
SN	non-primary alignments:	10906
SN	total length:	113899787	# ignores clipping
SN	total first fragment length:	113899787	# ignores clipping
SN	total last fragment length:	0	# ignores clipping
SN	bases mapped:	111430681	# ignores clipping
SN	bases mapped (cigar):	108297709	# more accurate
SN	bases trimmed:	0
SN	bases duplicated:	0
SN	mismatches:	19986130	# from NM fields
SN	error rate:	1.845480e-01	# mismatches / bases mapped (cigar)
SN	average length:	

In [22]:
 samtools view -c -F 260 $WKDIR/03.minimap2/Guppy_RedFlesh.bam 

20868


In [17]:
samtools view -c -F 0x10 $WKDIR/03.minimap2/Guppy_RedFlesh.bam "Chr09:35542701-35551900" #forward

121


In [18]:
samtools view -c -f 0x10 $WKDIR/03.minimap2/Guppy_RedFlesh.bam "Chr09:35542701-35551900" #reverse

75


In [5]:
 samtools view -c -F 260 $WKDIR/03.minimap2/Guppy_RedFlesh.bam "Chr09:35542701-35551878"

191


* Now we open the bam file of the mapping made after using albacore2 base calling and the bam file after using Guppy basecalling on ** IGV **

**  We dont appreciate an improvement on the Chr9:35543000 locus between albacore2 and Guppy basecalling** 

## Canu to be used to assemble MiniON targeted sequencing reads 

Comments on using Canu with targeted requencing reads coming from Pacbio 
* https://github.com/marbl/canu/issues/476

* Set genomeSize to your best guess of the combined size of the regions. You can probably be too high by an order of magnitude with no real problem; but too low would be somewhat bad. This impacts how corrected reads are computed, and how many corrected reads are computed.

* Once you get a feel for how they assemble, correctedErrorRate can be varied to refine how much assembly is done.

* Check the GFA outputs. If they're showing a big tangle, try dropping correctedErrorRate a bit. If unitigs look a lot like contigs, you've got a clean trusted assembly (and if they're big too, well, excellent).

In [68]:
mkdir -p $WKDIR/04.canu/new8
corrected=$WKDIR/Guppy_basecalling/04.canu/new8/Canu_Guppy.correctedReads.fasta.gz

In [82]:
bsub << EOF
#!/bin/bash
#BSUB -J Canu
#BSUB -o $WKDIR/04.canu/filter_contig/canu.out 
#BSUB -e $WKDIR/04.canu/filter_contig/canu.err
#BSUB -n 20
#BSUB -R "span[hosts=1]"w


thread=20
module load canu/1.7
canu -p Canu_Guppy -d $WKDIR/04.canu/filter_contig corMhapSensitivity=normal contigFilter="2 0 1.0 0.5 0" useGrid=false maxThreads=$thread genomeSize=400k -nanopore-raw $FQ  
#correctedErrorRate=0.25
EOF

Job <42899> is submitted to default queue <normal>.


*  -p option is to set the file name prefix of intermediate and output files, is mandatory. If -d is not supplied, canu will run in the current directory. Otherwise, Canu will create the assembly-directory and run in that directory.It is _not_ possible to run two different assemblies in the same directory.


*  For the genomeSize I estimated teh value for the total of the target region plus the rest of the regions that got sequenced and doubling that just to not be so low or understimating the size of amplicons that got sequenced here. 


*  The -s option will import a list of parameters from the supplied specification (‘spec’) file. These options can be useful if you want to correct reads once and try many different assemblies. We do exactly that in the Canu Quick Start. Additionally, suppling pre-corrected reads with -pacbio-corrected or -nanopore-corrected will run only the trimming (-trim) and assembling (-assemble) stages.


*  Additional parameters of canu https://canu.readthedocs.io/en/latest/parameter-reference.html#parameter-reference


** Make index to the assembly fasta file (.fai) **

In [45]:
module load samtools

In [46]:
samtools faidx $WKDIR/04.canu/new8/Canu_Guppy.contigs.fasta

In [47]:
grep ">" $WKDIR/04.canu/new8/Canu_Guppy.contigs.fasta

>tig00000001 len=145201 reads=45 covStat=409.58 gappedBases=no class=contig suggestRepeat=no suggestCircular=no
>tig00000003 len=59853 reads=29 covStat=87.00 gappedBases=no class=contig suggestRepeat=no suggestCircular=no
>tig00000010 len=3513 reads=12 covStat=-3.12 gappedBases=no class=contig suggestRepeat=no suggestCircular=no
>tig00000011 len=3026 reads=4 covStat=1.51 gappedBases=no class=contig suggestRepeat=no suggestCircular=no
>tig00000012 len=4633 reads=3 covStat=-0.33 gappedBases=no class=contig suggestRepeat=no suggestCircular=no
>tig00000015 len=5163 reads=5 covStat=4.74 gappedBases=no class=contig suggestRepeat=no suggestCircular=no


** If we compare the assembly from albacore basecalling notebook, we see that Guppy got one more contigs and larger ones **
#>tig00000001 len=130229 reads=87 covStat=351.14 gappedBases=no class=contig suggestRepeat=no suggestCircular=no
#>tig00000003 len=17146 reads=118 covStat=-47.97 gappedBases=no class=contig suggestRepeat=no suggestCircular=no
#>tig00000015 len=14585 reads=13 covStat=37.92 gappedBases=no class=contig suggestRepeat=no suggestCircular=no
#>tig00000017 len=16132 reads=15 covStat=13.60 gappedBases=no class=contig suggestRepeat=no suggestCircular=no
#>tig00000018 len=8006 reads=118 covStat=-75.37 gappedBases=no class=contig suggestRepeat=no suggestCircular=no
#>tig00000383 len=7197 reads=1 covStat=0.00 gappedBases=no class=contig suggestRepeat=no suggestCircular=no
#>tig00000384 len=7927 reads=45 covStat=-6.89 gappedBases=no class=contig suggestRepeat=no suggestCircular=no

# Map the generated contigs by Canu to the GD reference genome

In [70]:
Canu_assembled_contigs=$WKDIR/04.canu/filter_contig/Canu_Guppy.contigs.fasta
REF=/input/genomic/plant/Malus/Genome/GDDH_v1.1/Decompress/GDDH13_1-1_formatted.fasta 
canu_corrected=$WKDIR/Guppy_basecalling/04.canu/Canu_Guppy.correctedReads.fasta.gz
module load minimap2/2.9
module load samtools

In [37]:
mkdir -p $WKDIR/04.canu/minimap/Canu_contigs_vs_GD
mkdir -p $WKDIR/04.canu/minimap/Canu_contigs_vs_GD/logs

In [71]:
bsub << EOF
#!/bin/bash
#BSUB -J minimap2
#BSUB -o $WKDIR/04.canu/filter_contigcanu_contigs_vs_ref.out 
#BSUB -e $WKDIR/04.canu/filter_contig/canu_contigs_vs_ref.err
#BSUB -n 4
#BSUB -R "span[hosts=1]"

minimap2 -ax map-ont $REF $Canu_assembled_contigs | \
samtools view -Su - | samtools sort - -o $WKDIR/04.canu/filter_contig/Canu.Guppy.contigs.bam  
samtools index $WKDIR/04.canu/filter_contig/Canu.Guppy.contigs.bam
samtools depth $WKDIR/04.canu/filter_contig/Canu.Guppy.contigs.bam > $WKDIR/04.canu/filter_contig/Guppy_Canu_assembled_contigs.coverage
EOF


Job <42897> is submitted to default queue <normal>.


In [81]:
samtools view $WKDIR/04.canu/filter_contig/Canu.Guppy.contigs.bam |  awk '{if(length($10)>=1000) print$1,$2,$3,$4,$5,length($10)}' | grep "Chr09"

tig00000001 2048 Chr09 4815515 60 1323
tig00000001 2064 Chr09 4815515 60 1321
tig00000001 2064 Chr09 28299576 60 1334
tig00000001 2048 Chr09 28316322 60 1885
tig00000001 2064 Chr09 29602331 60 1250


# Map the generated unmapped contigs by Canu to the Ref GD genome

In [50]:
bsub << EOF
#!/bin/bash
#BSUB -J minimap2
#BSUB -o $WKDIR/04.canu/canu_unmappedcontigs_vs_ref.out 
#BSUB -e $WKDIR/04.canu/canu_unmappedcontigs_vs_ref.err
#BSUB -n 4
#BSUB -R "span[hosts=1]"

minimap2 -ax map-ont $REF $WKDIR/Guppy_basecalling/04.canu/Canu_Guppy.unassembled.fasta | \
samtools view -Su - | samtools sort - -o $WKDIR/04.canu/Canu_Guppy.unassembled.bam  
samtools index $WKDIR/04.canu/Canu_Guppy.unassembled.bam
samtools depth $WKDIR/04.canu/Canu_Guppy.unassembled.bam > $WKDIR/04.canu/Canu_Guppy.unassembled.coverage
EOF

Job <41429> is submitted to default queue <normal>.


In [1]:
module purge
module load samtools
samtools view  $WKDIR/04.canu/Canu_Guppy.unassembled.bam | awk '{if(length($10)>=1000) print$1,$2,$3,$4,$5,length($10)}' | grep "Chr09" | 

sort: cannot read: k1,1: No such file or directory
[E::hts_open_format] Failed to open file /04.canu/Canu_Guppy.unassembled.bam
samtools view: failed to open "/04.canu/Canu_Guppy.unassembled.bam" for reading: No such file or directory


: 2

## Alignment of canu corrected reads to GDv1.1 reference genome 

In [6]:
#mkdir $WKDIR/04.canu/minimap
#mkdir $WKDIR/04.canu/minimap/correct_reads_to_GD
mkdir $WKDIR/04.canu/minimap/correct_reads_to_GD/logs 

REF=/input/genomic/plant/Malus/Genome/GDDH_v1.1/Decompress/GDDH13_1-1_formatted.fasta 
canu_corrected=$WKDIR/Guppy_basecalling/04.canu/Canu_Guppy.correctedReads.fasta.gz 

module load minimap2/2.9
module load samtools

mkdir: cannot create directory ‘$WKDIR/Guppy_basecalling/04.canu/minimap/correct_reads_to_GD/logs’: File exists


** Minimap2 with default option to ONT reads **

In [13]:
bsub << EOF
#!/bin/bash
#BSUB -J minimap_corrected_canu_to_GD
#BSUB -o $WKDIR/04.canu/minimap/correct_reads_to_GD/logs/corr_vs_GD.out 
#BSUB -e $WKDIR/04.canu/minimap/correct_reads_to_GD/logs/corr_vs_GD.err
#BSUB -n 4
#BSUB -R "span[hosts=1]"

#minimap2 -ax map-ont $REF $canu_corrected | \
#samtools view -Su - | samtools sort - -o $WKDIR/04.canu/minimap/correct_reads_to_GD/Guppy_canu_corrected_vs_GDv1.1.bam  
#samtools index $WKDIR/04.canu/minimap/correct_reads_to_GD/Guppy_canu_corrected_vs_GDv1.1.bam
#samtools depth $WKDIR/04.canu/minimap/correct_reads_to_GD/Guppy_canu_corrected_vs_GDv1.1.bam > $WKDIR/04.canu/minimap/correct_reads_to_GD/Guppy_canu_corrected_vs_GDv1.1.coverage

module purge
module load samtools/1.9
bam=$WKDIR/Guppy_basecalling/04.canu/minimap/correct_reads_to_GD/Guppy_canu_corrected_vs_GDv1.1.bam
aqui=$WKDIR/Guppy_basecalling/04.canu/minimap/correct_reads_to_GD
samtools view -H $bam > $aqui/Guppy_canu_corrected_vs_GDv1.1.sam  #print header
samtools view -F 260 $bam | awk '$5>=40 {print$0}' >> $aqui/Guppy_canu_corrected_vs_GDv1.1.sam #select just those mapped reads with MQ>=40
samtools view -S -b -h $aqui/Guppy_canu_corrected_vs_GDv1.1.sam > $aqui/Guppy_canu_corrected_vs_GDv1.1_F260_q40.bam #convert to bam
samtools depth $aqui/Guppy_canu_corrected_vs_GDv1.1_F260_q40.bam > $aqui/  #compute coverage per bp

EOF




Job <376365> is submitted to default queue <normal>.


Check on IGV if this bam file has less errors than the one done with albacore2 also using Canu corrected reads

It did improved a lot the errors!!!

** Minimap2 with asm5 option to avoid off-targets **

In [17]:
bsub << EOF
#!/bin/bash
#BSUB -J minimap_corrected_canu_to_GD
#BSUB -o $WKDIR/04.canu/minimap/correct_reads_to_GD/logs/ASM5_corr_vs_GD.out 
#BSUB -e $WKDIR/04.canu/minimap/correct_reads_to_GD/logs/ASM5_corr_vs_GD.err
#BSUB -n 4
#BSUB -R "span[hosts=1]"

/workspace/hrarnc/GitHub/minimap2/minimap2 -cx asm5 $REF $canu_corrected | \
samtools view -Su - | samtools sort - -o $WKDIR/04.canu/minimap/correct_reads_to_GD/ASM5_Guppy_canu_corrected_vs_GDv1.1.bam  
samtools index $WKDIR/04.canu/minimap/correct_reads_to_GD/ASM5_Guppy_canu_corrected_vs_GDv1.1.bam
samtools depth $WKDIR/04.canu/minimap/correct_reads_to_GD/ASM5_Guppy_canu_corrected_vs_GDv1.1.bam > $WKDIR/04.canu/minimap/correct_reads_to_GD/ASM5_Guppy_canu_corrected_vs_GDv1.1.coverage
EOF

Job <27467> is submitted to default queue <normal>.


Like the case when basecaling with albacore2, asm5 doesn't produce a bam file for this data. It is too restringing function

In [18]:
samtools view $WKDIR/04.canu/minimap/correct_reads_to_GD/Guppy_canu_corrected_vs_GDv1.1.bam | awk '{if(length($10)>=6000) print$1,$2,$3,$4,$5,length($10)}'| grep "Chr09" | sort -u  

05118b93-67ce-4057-9520-2a92c9402fe7 0 Chr09 35542867 60 7745
05257628-753c-4003-a7d4-3fc704f52931 16 Chr09 35542879 60 7710
058d2785-d1f5-46e3-8c59-bb395f9e15dd 0 Chr09 35542912 60 7716
0614ca4f-7c97-43cb-b83f-bb9591672062 0 Chr09 35542870 60 7854
06cad05e-afb7-435f-bec4-6e8b5ef36855 0 Chr09 35542867 60 7771
07c7826c-3409-4508-aa0a-b1ddd60feda4 16 Chr09 35542952 60 7607
088eb005-9738-433f-94af-fbbed49ec710 16 Chr09 35542747 60 7926
0a645207-3dc4-4644-a0b4-f9229d1d482d 0 Chr09 35542867 60 7734
0a8e7ec0-e841-46fa-9ffe-6d7ab3e36a7a 16 Chr09 35542865 60 7889
0b8d1c64-2256-4cce-be52-2a838eaaff08 0 Chr09 35551901 60 42687
0b95a079-bee3-4b3d-8201-a888d9fd92dd 16 Chr09 35542867 60 7748
0d01b20c-acf8-415d-8b1e-34fe771a05a7 0 Chr09 35542867 60 7696
0f1f4969-3ee3-470a-927a-52b59a6e74ee 16 Chr09 35542768 60 7920
0ffcbf3e-a712-4021-b2c0-0fb0fd6245c5 0 Chr09 30202003 1 13536
10d4ec4e-2ca5-494f-b1da-a56ac6cd4412 0 Chr09 35542866 60 7853
14e696fa-8f21-4f68-ba73-88ffb4bb0306 0 Chr09 35542896 60 7680
1

In [22]:
awk '{if($3>=50) print$0}' $WKDIR/04.canu/minimap/correct_reads_to_GD/Guppy_canu_corrected_vs_GDv1.1.coverage | grep "Chr09" | wc -l

7810


In [7]:
awk '{if($3>=100) print$0}' $WKDIR/04.canu/minimap/correct_reads_to_GD/Guppy_canu_corrected_vs_GDv1.1.coverage | grep "Chr09" | head

Chr09	35542870	101
Chr09	35542871	109
Chr09	35542872	109
Chr09	35542873	109
Chr09	35542874	111
Chr09	35542875	112
Chr09	35542877	116
Chr09	35542878	118
Chr09	35542879	121
Chr09	35542880	122
grep: write error
awk: cmd. line:1: (FILENAME=$WKDIR/Guppy_basecalling/04.canu/minimap/correct_reads_to_GD/Guppy_canu_corrected_vs_GDv1.1.coverage FNR=3691890) fatal: print to "standard output" failed (Broken pipe)


# De novo assembly with Flye

In [257]:
module load flye/2.5

Conflict with existing Conda environment. Please unload/deactivate flye-2.5


In [195]:
mkdir $WKDIR/05.flye

In [269]:
albacore_assembly=$WKDIR/04.canu/Redflesh_Chr09_canu.contigs.fasta

In [270]:
canu_nanopolished=$WKDIR/05.nanopolish/polished_Redflesh_contig.fa

In [16]:
correct_canu=$WKDIR/Guppy_basecalling/04.canu/Canu_Guppy.correctedReads.fasta.gz 

** # NOTE:    We are feeding flye with a previous assembly I did with ON raw data base called with albacore2 plus the ONT raw \
data basecalled with guppy and corrected by canu. Canu wasn't producing a detent assembly with either the corrected canu guppy reads or from raw guppy reads. All the target reads were ending up as single contigs in those canu runs! **

In [303]:
bsub << EOF
#!/bin/bash
#BSUB -J Flye
#BSUB -o $WKDIR/05.flye/assembly-final/flye.out 
#BSUB -e $WKDIR/05.flye/assembly-final/flye.err
#BSUB -n 20
#BSUB -R "span[hosts=1]"
module purge
module load flye/2.5
flye --subassemblies $correct_canu $canu_nanopolished --threads 20 -g 150k -i 1 -o $WKDIR/05.flye/assembly-final 
EOF

Job <33767> is submitted to default queue <normal>.


Number of contigs on the assembly just generated

Contigs with more than 10 reads when assembled

In [301]:
grep -v "#" $WKDIR/05.flye/assembly-final/assembly_info.txt | wc -l 

571


In [14]:
awk '{if($3>=10) print$0}' $WKDIR/05.flye/assembly-final/assembly_info.txt


#seq_name	length	cov.	circ.	repeat	mult.	graph_path
contig_16	7915	86	-	+	70	*,16,*
contig_70	8029	71	-	+	58	*,70,*
contig_83	26263	15	-	+	15	83


The size of ** contig_16 and 70 make ** sense with Red Flesh locus, and also coverage

In [33]:
flye_assembly=$WKDIR/Guppy_basecalling/05.flye/assembly-final/assembly.fasta

In [214]:
mkdir $WKDIR/05.flye/minimap

In [13]:
bsub << EOF
#!/bin/bash
#BSUB -J Flye-assembly_vs_GD
#BSUB -o $WKDIR/05.flye/minimap/minimap.out
#BSUB -e $WKDIR/05.flye/minimap/minimap.err
#BSUB -n 4
#BSUB -R "span[hosts=1]"

module load samtools
module load minimap2/2.9
minimap2 -ax map-ont $REF $flye_assembly | \
samtools view -Su - | samtools sort - -o $WKDIR/05.flye/minimap/FINAL_contigs_vs_GD.bam  
samtools index $WKDIR/05.flye/minimap/FINAL_contigs_vs_GD.bam
samtools depth $WKDIR/05.flye/minimap/FINAL_contigs_vs_GD.bam > $WKDIR/05.flye/minimap/FINAL_contigs_vs_GD.coverage
EOF

Job <34263> is submitted to default queue <normal>.


In [26]:
module load samtools
samtools view $WKDIR/05.flye/minimap/FINAL_contigs_vs_GD.bam | awk '{if((length($10)>=5000)&&($4>=35542000)) print$1,$2,$3,$4,$5,length($10)}' | grep "Chr09" | grep "contig_16" 

contig_16 16 Chr09 35542721 60 7915


In [25]:
module load samtools
samtools view $WKDIR/05.flye/minimap/FINAL_contigs_vs_GD.bam | awk '{if(length($10)>=0) print$1,$2,$3,$4,$5,length($10)}' | grep "contig_70"  

contig_70 16 Chr09 35542725 60 8029


So contig_70 and contig_16 are hopefully the two haplotypes of the red flesh locus but regarding the author while being in Canberra said that they might not e phased. Maybe we can compare this with Illumina data to see of we get the same SNPs all the way and find out if they are phased.

** To confirme that we will extract this contigs in different fasta files and map the Guppy_canu corrected reads againts each contig **

In [28]:
samtools faidx $flye_assembly contig_70 > $WKDIR/05.flye/minimap/Contig_A_redflesh.fasta

In [4]:
samtools faidx $flye_assembly contig_16 > $WKDIR/05.flye/minimap/Contig_B_redflesh.fasta

In [15]:
contigA=$WKDIR/05.flye/minimap/Contig_A_redflesh.fasta
contigB=$WKDIR/05.flye/minimap/Contig_B_redflesh.fasta

In [17]:
bsub << EOF
#!/bin/bash
#BSUB -J minimap_guppy_vs_contigA
#BSUB -o $WKDIR/05.flye/minimap/minimap_contig_A.out
#BSUB -e $WKDIR/05.flye/minimap/minimap_contig_A.err
#BSUB -n 4
#BSUB -R "span[hosts=1]"

module load samtools
module load minimap2/2.9
minimap2 -ax map-ont $contigA $correct_canu | \
samtools view -Su - | samtools sort - -o $WKDIR/05.flye/minimap/Guppy_corrected_vs_ContigA.bam  
samtools index $WKDIR/05.flye/minimap/Guppy_corrected_vs_ContigA.bam 
samtools depth $WKDIR/05.flye/minimap/Guppy_corrected_vs_ContigA.bam  > $WKDIR/05.flye/minimap/Guppy_corrected_vs_ContigA.coverage
EOF

Job <34579> is submitted to default queue <normal>.


In [18]:
bsub << EOF
#!/bin/bash
#BSUB -J minimap_guppy_vs_contigB
#BSUB -o $WKDIR/05.flye/minimap/minimap_contig_A.out
#BSUB -e $WKDIR/05.flye/minimap/minimap_contig_A.err
#BSUB -n 4
#BSUB -R "span[hosts=1]"

module load samtools
module load minimap2/2.9
minimap2 -ax map-ont $contigB $correct_canu | \
samtools view -Su - | samtools sort - -o $WKDIR/05.flye/minimap/Guppy_corrected_vs_ContigB.bam  
samtools index $WKDIR/05.flye/minimap/Guppy_corrected_vs_ContigB.bam 
samtools depth $WKDIR/05.flye/minimap/Guppy_corrected_vs_ContigB.bam  > $WKDIR/05.flye/minimap/Guppy_corrected_vs_ContigB.coverage
EOF

Job <34580> is submitted to default queue <normal>.


In [5]:
awk '{if($3>=100) print$0}' $WKDIR/05.flye/minimap/Guppy_corrected_vs_ContigA.coverage | wc -l 

7727


In [6]:
awk '{if($3>=100) print$0}' $WKDIR/05.flye/minimap/Guppy_corrected_vs_ContigB.coverage | wc -l 

7718


Visualize the two haplotypes in IGV and print a screenshoot here

## Comparing contig A and contig B from Flye assembly by using nucmer 

In [18]:
cd $WKDIR/Guppy_basecalling/05.flye/minimap
mkdir nucmer_contigA_vs_contigB/logs
WD=$WKDIR/Guppy_basecalling/05.flye/minimap/nucmer_contigA_vs_contigB
contigA=$WKDIR/05.flye/minimap/Contig_A_redflesh.fasta
contigB=$WKDIR/05.flye/minimap/Contig_B_redflesh.fasta

mkdir: cannot create directory ‘nucmer_contigA_vs_contigB/logs’: File exists


In [52]:
bsub << EOF
#!/bin/bash
#BSUB -J Nucmer
#BSUB -o $WD/logs/nucmer.out
#BSUB -e $WD/logs/nucmer.err
#BSUB -n 1
#BSUB -R "span[hosts=1]"w

export GDFONTPATH=/usr/share/fonts/msttcore/
#/software/bioinformatics/MUMmer-3.23/nucmer -c 65 -l 65 --maxmatch -p $WD/nucmer_contigA_vs_contigB -o $contigA $contigB

#Then I filter the delta file for several similarity ranges (this step not needed here as the identity of the two contigs was higher than 95%)
#/software/bioinformatics/MUMmer-3.23/delta-filter -l 1000 -q -r -i 80 -1 $WD/nucmer_contigA_vs_contigB.delta > $WD/nucmer_contigA_vs_contigB_i80.delta

# Now ploting the contig in the y axis to see hpw the contig was put together and identify chimeric contigs

/software/bioinformatics/MUMmer-3.23/mummerplot -t png -s small -q contig_16 -p $WD/contigA_vs_contigB $WD/nucmer_contigA_vs_contigB.delta

/software/bioinformatics/MUMmer-3.23/dnadiff --prefix $WD/contigA_vs_contigB.dnadiff \
$contigA $contigB


EOF

Job <376474> is submitted to default queue <normal>.


In [25]:
head -n 40 $WD/contigA_vs_contigB.dnadiff.report


$WKDIR/Guppy_basecalling/05.flye/minimap/Contig_A_redflesh.fasta $WKDIR/Guppy_basecalling/05.flye/minimap/Contig_B_redflesh.fasta
NUCMER

                               [REF]                [QRY]
[Sequences]
TotalSeqs                          1                    1
AlignedSeqs               1(100.00%)           1(100.00%)
UnalignedSeqs               0(0.00%)             0(0.00%)

[Bases]
TotalBases                      8029                 7915
AlignedBases            7906(98.47%)         7892(99.71%)
UnalignedBases            123(1.53%)            23(0.29%)

[Alignments]
1-to-1                             2                    2
TotalLength                     7906                 7892
AvgLength                    3953.00              3946.00
AvgIdentity                    97.96                97.96

M-to-M                             2                    2
TotalLength                     7906                 7892
AvgLength                    3953.00              3946.00
AvgIdentity   

In [29]:
#Alignment between Conig A and B
/software/bioinformatics/MUMmer-3.23/show-aligns $WD/contigA_vs_contigB.dnadiff.delta \
'contig_70' 'contig_16' > $WD/ref_qry.aligns


In [30]:
head $WD/ref_qry.aligns

$WKDIR/Guppy_basecalling/05.flye/minimap/Contig_A_redflesh.fasta $WKDIR/Guppy_basecalling/05.flye/minimap/Contig_B_redflesh.fasta

-- Alignments between contig_70 and contig_16

-- BEGIN alignment [ +1 4 - 5918 | +1 1 - 5910 ]


4          cgaagacttaactctttcggttgcagatccaagccttcctatcaaactt
1          cgaagacttaactctttcggttgcagatccaagccttcctatcaaactt


## Note: when I checked the alignment I realised that the two contigs are the reverse and complement version so I am going to do reverse and complement of the fasta file of the contigs and repeat the above

In [31]:
#It will be easier to identify the R6 motif and to compare it with the Canu- asembled contigs before and after Nanopolish

In [34]:
flye_assembly=$WKDIR/Guppy_basecalling/05.flye/assembly-final/assembly.fasta

In [45]:
samtools faidx -i $flye_assembly contig_70 > $WKDIR/05.flye/minimap/rev_comp_Contig_A_redflesh.fasta #-i gaves reverse complement

In [46]:
samtools faidx -i $flye_assembly contig_16 > $WKDIR/05.flye/minimap/rev_comp_Contig_B_redflesh.fasta #-i gaves reverse complement

In [47]:
WD=$WKDIR/Guppy_basecalling/05.flye/minimap/nucmer_contigA_vs_contigB
contigA_rev_comp=$WKDIR/05.flye/minimap/rev_comp_Contig_A_redflesh.fasta
contigB_rev_comp=$WKDIR/05.flye/minimap/rev_comp_Contig_B_redflesh.fasta

In [55]:
bsub << EOF
#!/bin/bash
#BSUB -J Nucmer
#BSUB -o $WD/logs/nucmer.out
#BSUB -e $WD/logs/nucmer.err
#BSUB -n 1
#BSUB -R "span[hosts=1]"w

export GDFONTPATH=/usr/share/fonts/msttcore/
#/software/bioinformatics/MUMmer-3.23/nucmer -c 65 -l 65 --maxmatch -p $WD/rev_comp_nucmer_contigA_vs_contigB -o $contigA_rev_comp $contigB_rev_comp 

#Then I filter the delta file for several similarity ranges (this step not needed here as the identity of the two contigs was higher than 95%)
#/software/bioinformatics/MUMmer-3.23/delta-filter -l 1000 -q -r -i 80 -1 $WD/rev_comp_nucmer_contigA_vs_contigB.delta > $WD/rev_comp_nucmer_contigA_vs_contigB_i80.delta

# Now ploting the contig in the y axis to see hpw the contig was put together and identify chimeric contigs

#/software/bioinformatics/MUMmer-3.23/mummerplot -t png -s small -q contig_16/rc -p $WD/rev_comp_contigA_vs_contigB $WD/rev_comp_nucmer_contigA_vs_contigB.delta

/software/bioinformatics/MUMmer-3.23/dnadiff --prefix $WD/rev_comp_contigA_vs_contigB.dnadiff \
$contigA_rev_comp $contigB_rev_comp


EOF

Job <376477> is submitted to default queue <normal>.


In [56]:
head -n 40 $WD/rev_comp_contigA_vs_contigB.dnadiff.report

$WKDIR/Guppy_basecalling/05.flye/minimap/rev_comp_Contig_A_redflesh.fasta $WKDIR/Guppy_basecalling/05.flye/minimap/rev_comp_Contig_B_redflesh.fasta
NUCMER

                               [REF]                [QRY]
[Sequences]
TotalSeqs                          1                    1
AlignedSeqs               1(100.00%)           1(100.00%)
UnalignedSeqs               0(0.00%)             0(0.00%)

[Bases]
TotalBases                      8029                 7915
AlignedBases            7906(98.47%)         7892(99.71%)
UnalignedBases            123(1.53%)            23(0.29%)

[Alignments]
1-to-1                             2                    2
TotalLength                     7906                 7892
AvgLength                    3953.00              3946.00
AvgIdentity                    97.96                97.96

M-to-M                             2                    2
TotalLength                     7906                 7892
AvgLength                    3953.00              3946

In [57]:
#Alignment between Conig A and B
/software/bioinformatics/MUMmer-3.23/show-aligns $WD/rev_comp_contigA_vs_contigB.dnadiff.delta \
'contig_70/rc' 'contig_16/rc' > $WD/rev_comp_ref_qry.aligns


In [58]:
head $WD/rev_comp_ref_qry.aligns

$WKDIR/Guppy_basecalling/05.flye/minimap/rev_comp_Contig_A_redflesh.fasta $WKDIR/Guppy_basecalling/05.flye/minimap/rev_comp_Contig_B_redflesh.fasta

-- Alignments between contig_70/rc and contig_16/rc

-- BEGIN alignment [ +1 1 - 1991 | +1 5 - 1986 ]


1          aggggagcggagaggatgggctctcagcccggaccgaaccatttccacc
5          aggggagcggagaggatgggctctcagcccggaccgaaccatttccacc


In [60]:
samtools faidx $WKDIR/05.flye/minimap/rev_comp_Contig_A_redflesh.fasta "contig_70/rc:1991-2112" > $WD/insertion_contig_70_rc:1991-2112.fasta

In [61]:
head $WD/insertion_contig_70_rc:1991-2112.fasta

>contig_70/rc:1991-2112
GACTGGTAGCTATTAACAAGTTAGACTGGTTAGACTGGTAGCTATTAACAAGTTAGACTG
GTAGCTATTAACAACTGGTAGCTATTAACAAGTTAGACTGGTAGCTATTAACAAGTTAGA
CT


In [63]:
samtools faidx $WKDIR/05.flye/minimap/rev_comp_Contig_B_redflesh.fasta "contig_16/rc:1986-2006" > $WD/insertion_contig_16_rc:1986-2006.fasta

In [64]:
head  $WD/insertion_contig_16_rc:1986-2006.fasta

>contig_16/rc:1986-2006
GCTTTTCTATATATCGAGTGT


### Alignment of Canu-corrected reads againts Contig 16 and 70 to see if each kind of read (with and without the motif of R6) map correctly to each haplotype-contig

- Previously I did this already but contigs were not reversed complemented so I am repeating it here with reversed complemented contigs and \
  and applying the -F 260 and MQ 40

In [ ]:
WD=$WKDIR/Guppy_basecalling/05.flye/minimap
contigA_rev_comp=$WKDIR/05.flye/minimap/rev_comp_Contig_A_redflesh.fasta
contigB_rev_comp=$WKDIR/05.flye/minimap/rev_comp_Contig_B_redflesh.fasta 
correct_canu=$WKDIR/Guppy_basecalling/04.canu/Canu_Guppy.correctedReads.fasta
WKDIR=$WKDIR/Guppy_basecalling

In [87]:
# for contig 70 rc (A)

module load samtools/1.9
module load minimap2/2.9
#minimap2 -ax map-ont  $contigA_rev_comp $correct_canu > $WD/Guppy_corrected_vs_ContigA.bam
#samtools view -H $WD/Guppy_corrected_vs_ContigA.bam > $WD/Guppy_canu_corrected_vs_rev_comp_Contig_A.sam
#samtools view -F 260 $WD/Guppy_corrected_vs_ContigA.bam | awk '$5>=40 {print$0}' >> $WD/Guppy_canu_corrected_vs_rev_comp_Contig_A.sam
#samtools view -S -b -h $WD/Guppy_canu_corrected_vs_rev_comp_Contig_A.sam | samtools sort - > $WD/Guppy_canu_corrected_vs_rev_comp_Contig_A_F260_Q40.bam #convert to bam    
#samtools depth $WD/Guppy_canu_corrected_vs_rev_comp_Contig_A_F260_Q40.bam > $WD/Guppy_canu_corrected_vs_rev_comp_Contig_A_F260_Q40.coverage
samtools index $WD/Guppy_canu_corrected_vs_rev_comp_Contig_A_F260_Q40.bam

In [89]:
samtools faidx $contigA_rev_comp
samtools faidx $contigB_rev_comp

In [88]:
# For contig 16 (B)
module load samtools/1.9
module load minimap2/2.9
#minimap2 -ax map-ont $contigB_rev_comp $correct_canu > $WD/Guppy_corrected_vs_ContigB.bam
#samtools view -H $WD/Guppy_corrected_vs_ContigB.bam > $WD/Guppy_canu_corrected_vs_rev_comp_Contig_B.sam
#samtools view -F 260 $WD/Guppy_corrected_vs_ContigB.bam | awk '$5>=40 {print$0}' >> $WD/Guppy_canu_corrected_vs_rev_comp_Contig_B.sam
#samtools view -S -b -h $WD/Guppy_canu_corrected_vs_rev_comp_Contig_B.sam | samtools sort - > $WD/Guppy_canu_corrected_vs_rev_comp_Contig_B_F260_Q40.bam #convert to bam    
#samtools depth $WD/Guppy_canu_corrected_vs_rev_comp_Contig_B_F260_Q40.bam > $WD/Guppy_canu_corrected_vs_rev_comp_Contig_B_F260_Q40.coverage
samtools index $WD/Guppy_canu_corrected_vs_rev_comp_Contig_B_F260_Q40.bam

# Generate Coverage plot for each of these Flye contigs

In [82]:
module load bedtools/2.27.1

In [84]:
#extract coverage for forward strand for contig_70 (A)
bedtools coverage -s -d -a $WD/contig_70.bed -b $WD/Guppy_canu_corrected_vs_rev_comp_Contig_A_F260_Q40.bam > $WD/bedtools_cov_Forward_strand_Contig70_A.tsv
#extract coverage for reverse strand for contig_70 (A)
bedtools coverage -S -d -a $WD/contig_70.bed -b $WD/Guppy_canu_corrected_vs_rev_comp_Contig_A_F260_Q40.bam > $WD/bedtools_cov_Reverse_strand_Contig70_A.tsv

In [85]:
#extract coverage for forward strand for contig_16 (B)
bedtools coverage -s -d -a $WD/contig_16.bed -b $WD/Guppy_canu_corrected_vs_rev_comp_Contig_B_F260_Q40.bam > $WD/bedtools_cov_Forward_strand_Contig16_B.tsv
#extract coverage for reverse strand for contig_16 (B)
bedtools coverage -S -d -a $WD/contig_16.bed -b $WD/Guppy_canu_corrected_vs_rev_comp_Contig_B_F260_Q40.bam > $WD/bedtools_cov_Reverse_strand_Contig16_B.tsv

* Then we used this files as input in this R script found at:
* $WKDIR/Guppy_basecalling/05.flye/minimap/Coverage_plots_bedtools_corrected_reads_against_Flye_contigs_70_16.R

In [ ]:
# Extract reads with the insertion and deletion from each of the mapping files 
samtools view -c Guppy_canu_corrected_vs_rev_comp_Contig_B_F260_Q40.bam
samtools view Guppy_canu_corrected_vs_rev_comp_Contig_B_F260_Q40.bam | cut -f 1,6 > CIGAR_Guppy_canu_corrected_vs_rev_comp_Contig_B_F260_Q40.txt
grep "99I" CIGAR_Guppy_canu_corrected_vs_rev_comp_Contig_B_F260_Q40.txt | awk '{print$1}' > reads_99I_ContigB.txt
wc -l reads_99I_ContigB.txt
samtools view Guppy_canu_corrected_vs_rev_comp_Contig_A_F260_Q40.bam | cut -f 1,6 > CIGAR_Guppy_canu_corrected_vs_rev_comp_Contig_A_F260_Q40.txt
grep "99D" CIGAR_Guppy_canu_corrected_vs_rev_comp_Contig_A_F260_Q40.txt | awk '{print$1}' > reads_99D_ContigA.txt
wc -l reads_99D_ContigA.txt

In [90]:
module purge
module load bambam/1.4

In [109]:
for i in {70..99}
do
grep ""$i"D" CIGAR_Guppy_canu_corrected_vs_rev_comp_Contig_A_F260_Q40.txt | awk '{print$1}' | wc -l 
done

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
65


In [110]:
for i in {70..99}
do
grep ""$i"I" CIGAR_Guppy_canu_corrected_vs_rev_comp_Contig_B_F260_Q40.txt | awk '{print$1}' | wc -l 
done

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
7
0
48


In [111]:
# so there is Insertion of 99 and 97
grep "97I" CIGAR_Guppy_canu_corrected_vs_rev_comp_Contig_B_F260_Q40.txt | awk '{print$1}' >> reads_99I_ContigB.txt

In [112]:
wc -l  reads_99I_ContigB.txt

55 reads_99I_ContigB.txt


In [ ]:
#combined 99I and 99D
cat reads_99I_ContigB.txt reads_99D_ContigA.txt >> combined_99I_99D.txt

In [120]:
head combined_99I_99D.txt

b5c4b76e-23a7-4604-9f22-2a4c95fdb73a
6d9ff872-668d-4c1d-9b7e-3306f8a08eaf
58f232ae-dcd8-441d-93c1-e6a1a2774ef0
eb4edd26-af9d-40c4-8f40-9579f5fc1693
7327e74b-2ebe-472e-a0f9-3d84e708ab44
de65a9de-e0e0-453a-912e-3b0266b0aa2a
baf8e988-83d2-42ae-ab17-e759f6eb394c
f626d1af-41d4-49b7-a036-7bc4f00a6584
088eb005-9738-433f-94af-fbbed49ec710
af060d27-97a6-4441-8937-b32b30db1dd6


In [154]:
#reads that dont group neither in 99I or 99D
grep -v -f "combined_99I_99D.txt" CIGAR_Guppy_canu_corrected_vs_rev_comp_Contig_B_F260_Q40.txt | awk '{print$1}' > NON_99I_99D.txt

12


In [169]:
head reads_99I_ContigB.txt

b5c4b76e-23a7-4604-9f22-2a4c95fdb73a
6d9ff872-668d-4c1d-9b7e-3306f8a08eaf
58f232ae-dcd8-441d-93c1-e6a1a2774ef0
eb4edd26-af9d-40c4-8f40-9579f5fc1693
7327e74b-2ebe-472e-a0f9-3d84e708ab44
de65a9de-e0e0-453a-912e-3b0266b0aa2a
baf8e988-83d2-42ae-ab17-e759f6eb394c
f626d1af-41d4-49b7-a036-7bc4f00a6584
088eb005-9738-433f-94af-fbbed49ec710
af060d27-97a6-4441-8937-b32b30db1dd6


In [ ]:
#the name of reads in Canu_Guppy.correctedReads.fasta is to long and then the matching wont work so we need to chanhe the ID in that fasta file

WD2=$WKDIR/Guppy_basecalling/04.canu
awk -F " " '{ if ($0 ~ /^>/) { print $1;} else { print $0}}' $WD2/Canu_Guppy.correctedReads.fasta > $WD2/Canu_Guppy.correctedReads_short_ID.fasta
grep ">" $WD2/Canu_Guppy.correctedReads_short_ID.fasta | head

In [ ]:
#Extract reads 
awk -F'>' 'NR==FNR{ids[$0]; next} NF>1{f=($2 in ids)} f' reads_99D_ContigA.txt $WKDIR/Guppy_basecalling/04.canu/Canu_Guppy.correctedReads_short_ID.fasta > reads_99D_ContigA.fasta
awk -F'>' 'NR==FNR{ids[$0]; next} NF>1{f=($2 in ids)} f' reads_99I_ContigB.txt $WKDIR/Guppy_basecalling/04.canu/Canu_Guppy.correctedReads_short_ID.fasta > reads_99I_ContigB.fasta #55
awk -F'>' 'NR==FNR{ids[$0]; next} NF>1{f=($2 in ids)} f' NON_99I_99D.txt $WKDIR/Guppy_basecalling/04.canu/Canu_Guppy.correctedReads_short_ID.fasta > NON_99I_99D.fasta

In [ ]:
non_99I_non_99D=$WKDIR/Guppy_basecalling/05.flye/minimap/NON_99I_99D.fasta
reads_99I_ContigB=$WKDIR/Guppy_basecalling/05.flye/minimap/reads_99I_ContigB.fasta
reads_99D_ContigA=$WKDIR/Guppy_basecalling/05.flye/minimap/reads_99D_ContigA.fasta

In [ ]:
module load samtools/1.9
module load minimap2/2.9
minimap2 -ax map-ont $contigB_rev_comp $non_99I_non_99D > $WD/non_99I_non_99D_vscontigB.bam
samtools view -H $WD/non_99I_non_99D_vscontigB.bam > $WD/non_99I_non_99D_vscontigB.sam
samtools view -F 260 $WD/non_99I_non_99D_vscontigB.bam | awk '$5>=40 {print$0}' >> $WD/non_99I_non_99D_vscontigB.sam
samtools view -S -b -h$WD/non_99I_non_99D_vscontigB.sam | samtools sort - > $WD/non_99I_non_99D_vs_Contig_B_F260_Q40.bam #convert to bam    
samtools depth $WD/non_99I_non_99D_vs_Contig_B_F260_Q40.bam >$WD/non_99I_non_99D_vs_Contig_B_F260_Q40.coverage
samtools index $WD/non_99I_non_99D_vs_Contig_B_F260_Q40.bam

In [ ]:
#Once this was bam was produced I went to IGV and manually visualized each of these unclassifed reads, copy to clipboard ther deatails
#and add each each to each proper haplotype, either to file reads_99I_ContigB.txt if it had the insertion (110bp) or if not to reads_99D_ContigA.txt


In [ ]:
#So now we need to modify these fasta files to add these additional reads:
#So we need to repeat this command
#Extract reads 
awk -F'>' 'NR==FNR{ids[$0]; next} NF>1{f=($2 in ids)} f' reads_99D_ContigA.txt $WKDIR/Guppy_basecalling/04.canu/Canu_Guppy.correctedReads_short_ID.fasta > reads_99D_ContigA.fasta
awk -F'>' 'NR==FNR{ids[$0]; next} NF>1{f=($2 in ids)} f' reads_99I_ContigB.txt $WKDIR/Guppy_basecalling/04.canu/Canu_Guppy.correctedReads_short_ID.fasta > reads_99I_ContigB.fasta #

In [ ]:
#Mapping against contig A _contains the INSERTION
module load samtools/1.9
module load minimap2/2.9
minimap2 -ax map-ont rev_comp_Contig_A_redflesh.fasta reads_99I_ContigB.fasta > reads_99I_vs_contigA.bam
samtools view -H reads_99I_vs_contigA.bam > reads_99I_vs_contigA.sam
samtools view -F 260 reads_99I_vs_contigA.bam | awk '$5>=40 {print$0}' >> reads_99I_vs_contigA.sam
samtools view -S -b -h  reads_99I_vs_contigA.sam | samtools sort - > reads_99I_vs_contigA_F260_Q40.bam #convert to bam    
samtools depth reads_99I_vs_contigA_F260_Q40.bam > reads_99I_vs_contigA_F260_Q40.bam.coverage
samtools index reads_99I_vs_contigA_F260_Q40.bam

In [ ]:
#Mapping against contig B _contains the DELETION
module load samtools/1.9
module load minimap2/2.9
minimap2 -ax map-ont rev_comp_Contig_B_redflesh.fasta reads_99D_ContigA.fasta > reads_99D_vs_contigB.bam
samtools view -H reads_99D_vs_contigB.bam > reads_99D_vs_contigB.sam
samtools view -F 260 reads_99D_vs_contigB.bam | awk '$5>=40 {print$0}' >> reads_99D_vs_contigB.sam
samtools view -S -b -h  reads_99D_vs_contigB.sam | samtools sort - > reads_99D_vs_contigB_F260_Q40.bam #convert to bam    
samtools depth reads_99D_vs_contigB_F260_Q40.bam > reads_99D_vs_contigB_F260_Q40.bam.coverage
samtools index reads_99D_vs_contigB_F260_Q40.bam

In [ ]:
#AIM: we have a file of SNPs between de novo assembled by Flye, contig A and contig B 
#how many of those are present between contig B (like reference allele or non red) and apple ref genome, these will be ONT errors or could 
#still be errors in the reference
#how many of those are present at the vcf file done with re-seq illumian data on this same individual? This will be SNPs specific to the red 
#flesh allele against the ref

snsp_bt_A_B=$WKDIR/Guppy_basecalling/05.flye/minimap/nucmer_contigA_vs_contigB/rev_comp_contigA_vs_contigB.dnadiff.snps

#Get the locus from reference genome to use it a short reference to compare to contig B (non redflesh de novo haplotig)

module load samtools/1.9
cd $WKDIR/Guppy_basecalling/05.flye/minimap/nucmer_contigA_vs_contigB

samtools faidx /input/genomic/plant/Malus/Genome/GDDH_v1.1/Decompress/GDDH13_1-1_formatted.fasta Chr09:35542721-35550686 > target.fasta

/software/bioinformatics/MUMmer-3.23/dnadiff --prefix rev_comp_contigB_vs_target.dnadiff target.fasta $WKDIR/Guppy_basecalling/05.flye/minimap/rev_comp_Contig_B_redflesh.fasta

# Get the SNPs that are common between contig B(16) aganisnt contig A(70) and between contig A (non red) against the reference: this are likely 
#errors of ONT or ref errors

awk -F'\t' 'NR==FNR{c[$4]++;next};c[$4]' rev_comp_contigB_vs_target.dnadiff.snps rev_comp_contigA_vs_contigB.dnadiff.snps > Error_ONT_or_in_ref.snps


#Now do the other way around. Check Snps between red allele (contig A) and ref

/software/bioinformatics/MUMmer-3.23/dnadiff --prefix rev_comp_contigA_vs_target.dnadiff target.fasta $WKDIR/Guppy_basecalling/05.flye/minimap/rev_comp_Contig_A_redflesh.fasta

awk -F'\t' 'NR==FNR{c[$1]++;next};c[$1]' rev_comp_contigA_vs_target.dnadiff.snps rev_comp_contigB_vs_target.dnadiff.snps > common_SNPs_contig_a_b_vs_ref.snps
awk -F'\t' 'FNR==NR {a[$1]++; next} !a[$1]' rev_comp_contigB_vs_target.dnadiff.snps rev_comp_contigA_vs_target.dnadiff.snps > uniq_snps_contigA_ref.snps
awk -F'\t' 'FNR==NR {a[$1]++; next} !a[$1]' rev_comp_contigA_vs_target.dnadiff.snps rev_comp_contigB_vs_target.dnadiff.snps > uniq_snps_contigB_ref.snps